In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('patrol_dpr')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the current data from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_patrol'

In [10]:
patrol_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['patrol_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [11]:
sql_cols = list(patrol_sql.columns.values)

In [12]:
patrol_sql.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough


In [13]:
hash_rows(patrol_sql, exclude_cols = ['encounter_timestamp'], hash_name = 'row_hash')

### Read the site reference list from SQL

In [14]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [15]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the latest data from Google Sheets

In [16]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [17]:
sheet = client.open('COMBINED Patrol Reporting Responses')

In [18]:
ws = sheet.worksheet('MASTER')

In [19]:
#Read the worksheet as a data frame, rename the columns and subset the columns to only include those
#in the column list
patrol = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [20]:
patrol.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,park_division,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdassist,sd_pdcontact,sd_comments,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
1,5/29/2020 14:52:46,5/29/2020 12:30:00,Spirit Playground,NaN,Rangers,Nadilyn,Beato,Steven,Rehm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
2,5/29/2020 14:52:43,5/29/2020 9:30:00,Morningside Park-Morningside Playground,NaN,PEP,charles,argento,Karuna,Kaufman,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
3,5/29/2020 14:51:38,5/29/2020 14:50:00,Midland Playground,NaN,Rangers,Elizabeth,colon,Amanda,Seaquist,Joseph,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
4,5/29/2020 14:50:37,5/29/2020 11:50:00,Crocheron Park-Buz O' Rourke Playground,NaN,Rangers,Nadilyn,Beato,Steven,Beato,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
5,5/29/2020 14:48:36,5/29/2020 11:45:00,Crocheron Park,NaN,Rangers,Nadilyn,Beato,Steven,Rehm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [21]:
yesno = ['closed_education', 'closed_outcome', 'closed_summonsissued', 'closed_pdassist',
         'closed_pdcontact', 'sd_summonsissued', 'sd_pdassist', 'sd_pdcontact']

In [22]:
yesno_cols(patrol, yesno)

In [23]:
#Remove any rows with no data, presumably these are rows with no timestamp
patrol = patrol[patrol['encounter_timestamp'].notna()]

In [24]:
patrol = patrol.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [25]:
hash_rows(patrol, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

In [26]:
patrol_deltas = (check_deltas(new_df = patrol, old_df = patrol_sql, on = ['site_id', 'encounter_timestamp'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [27]:
patrol_inserts = patrol_deltas[patrol_deltas['dml_verb'] == 'I'][sql_cols]

In [28]:
patrol_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,park_division,visit_reason,firstname_1,lastname_1,firstname_2,lastname_2,...,sd_patronscomplied,sd_patronsnocomply,sd_amenity,summonscount_a01,summonscount_a03,summonscount_a04,summonscount_a22,other_summonstype,other_summonscount,borough
0,5/29/2020 14:52:46,5/29/2020 12:30:00,Q429,NaN,Rangers,Standard Patrol,Nadilyn,Beato,Steven,Rehm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
1,5/29/2020 14:52:43,5/29/2020 9:30:00,M056-02,NaN,PEP,Standard Patrol,charles,argento,Karuna,Kaufman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
2,5/29/2020 14:51:38,5/29/2020 14:50:00,R038,NaN,Rangers,Standard Patrol,Elizabeth,colon,Amanda,Seaquist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staten Island
3,5/29/2020 14:50:37,5/29/2020 11:50:00,Q012-01,NaN,Rangers,Standard Patrol,Nadilyn,Beato,Steven,Beato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens
4,5/29/2020 14:48:36,5/29/2020 11:45:00,Q012,NaN,Rangers,Standard Patrol,Nadilyn,Beato,Steven,Rehm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Queens


In [52]:
import datetime

In [90]:
def get_datetime(row,col):
    try:
        return pd.to_datetime(row[col],infer_datetime_format=True)
    except:
        print(row.name)

In [91]:
test = patrol.copy()

In [92]:
datetime_col = 'encounter_datetime'
test[datetime_col] = test.apply(lambda row: get_datetime(row, datetime_col),axis=1)

15770


In [94]:
patrol.iloc[15770]

encounter_timestamp                                    4/19/2020 18:03:56
encounter_datetime                                     4/19/0202 14:36:00
site_id                                                              Q400
location_adddesc                                                      NaN
park_division                                                     Rangers
visit_reason                                              Standard Patrol
firstname_1                                                          Doug
lastname_1                                                         Kenney
firstname_2                                                        Leslie
lastname_2                                                        Camacho
firstname_3                                                           NaN
lastname_3                                                            NaN
patrol_method                                                      Mobile
encounter_type                        

In [74]:
for i,v in test['encounter_datetime'].iteritems():
    if type(v) != 'pandas._libs.tslibs.timestamps.Timestamp'

{0: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{1: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{2: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{4: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{5: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{7: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{8: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{10: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{11: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{12: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{14: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{15: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{17: <class 'pandas._libs.tslibs.timestam

{3326: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3327: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3328: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3329: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3330: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3331: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3332: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3333: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3334: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3335: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3336: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3337: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3338: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3339: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3340: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3341: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{3342: <class 'pandas._libs.tslibs.timestamps.Timestamp'

{6488: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6489: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6490: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6491: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6492: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6493: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6494: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6495: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6496: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6497: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6498: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6499: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6500: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6501: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6502: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6503: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{6504: <class 'pandas._libs.tslibs.timestamps.Timestamp'

{9578: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9579: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9580: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9581: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9582: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9583: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9584: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9585: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9586: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9587: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9588: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9589: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9590: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9591: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9592: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9593: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{9594: <class 'pandas._libs.tslibs.timestamps.Timestamp'

{13014: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13015: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13016: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13017: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13018: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13019: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13020: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13021: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13022: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13023: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13024: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13025: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13026: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13027: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13028: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13029: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{13030: <class 'pandas._libs.tslibs.time

{16520: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16521: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16522: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16523: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16524: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16525: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16526: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16527: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16528: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16529: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16530: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16531: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16532: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16533: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16534: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16535: <class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{16536: <class 'pandas._libs.tslibs.time

In [37]:
x = list(test.unique())

In [50]:
print(x)

['5/29/2020 12:30:00', '5/29/2020 9:30:00', '5/29/2020 14:50:00', '5/29/2020 11:50:00', '5/29/2020 11:45:00', '5/29/2020 14:47:00', '5/29/2020 14:43:00', '5/29/2020 14:38:00', '5/29/2020 14:31:00', '5/29/2020 14:28:00', '5/29/2020 14:24:00', '5/29/2020 14:37:00', '5/28/2020 14:32:00', '5/28/2020 14:15:00', '5/29/2020 14:10:00', '5/28/2020 14:10:00', '5/29/2020 14:05:00', '5/28/2020 13:53:00', '5/29/2020 13:48:00', '5/29/2020 11:10:00', '5/28/2020 13:20:00', '5/29/2020 10:55:00', '5/29/2020 10:14:00', '5/29/2020 13:14:00', '5/28/2020 1:08:00', '5/29/2020 10:08:00', '5/29/2020 13:04:00', '5/29/2020 10:02:00', '5/28/2020 0:12:00', '5/29/2020 13:00:00', '5/29/2020 9:50:00', '5/29/2020 11:35:00', '5/29/2020 9:45:00', '5/28/2020 11:22:00', '5/29/2020 9:40:00', '5/29/2020 13:35:00', '5/29/2020 9:35:00', '5/29/2020 11:25:00', '5/29/2020 11:20:00', '5/28/2020 11:15:00', '5/28/2020 10:10:00', '5/29/2020 13:50:00', '5/29/2020 11:33:00', '5/29/2020 11:30:00', '5/29/2020 14:09:00', '5/29/2020 14:00

In [51]:
test.dtypes()

TypeError: 'numpy.dtype' object is not callable

In [47]:
for v in x:
    print(v == '...')

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [32]:
test.to_sql('tbl_dpr_patrol', engine, index = False, if_exists = 'append')

DataError: (pyodbc.DataError) ('22007', '[22007] [Microsoft][ODBC SQL Server Driver][SQL Server]The conversion of a nvarchar data type to a datetime data type resulted in an out-of-range value. (242) (SQLExecDirectW); [22007] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')
[SQL: INSERT INTO tbl_dpr_patrol (encounter_datetime) VALUES (?)]
[parameters: (('5/29/2020 12:30:00',), ('5/29/2020 9:30:00',), ('5/29/2020 14:50:00',), ('5/29/2020 11:50:00',), ('5/29/2020 11:45:00',), ('5/29/2020 14:47:00',), ('5/29/2020 14:43:00',), ('5/29/2020 14:43:00',)  ... displaying 10 of 18318 total bound parameter sets ...  ('4/14/2020 19:24:00',), ('4/14/2020 20:00:00',))]
(Background on this error at: http://sqlalche.me/e/9h9h)

In [48]:
patrol_inserts.to_sql('tbl_dpr_patrol', engine, index = False, if_exists = 'append')

DataError: (pyodbc.DataError) ('22007', '[22007] [Microsoft][ODBC SQL Server Driver][SQL Server]The conversion of a nvarchar data type to a datetime data type resulted in an out-of-range value. (242) (SQLExecDirectW); [22007] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')
[SQL: INSERT INTO tbl_dpr_patrol (encounter_timestamp, encounter_datetime, site_id, location_adddesc, park_division, visit_reason, firstname_1, lastname_1, firstname_2, lastname_2, firstname_3, lastname_3, patrol_method, encounter_type, closed_amenity, closed_patroncount, closed_education, closed_outcome, closed_summonsissued, closed_pdassist, closed_pdcontact, closed_comments, sd_summonsissued, sd_pdassist, sd_pdcontact, sd_comments, sd_patronscomplied, sd_patronsnocomply, sd_amenity, summonscount_a01, summonscount_a03, summonscount_a04, summonscount_a22, other_summonstype, other_summonscount, borough) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?

In [ ]:
patrol_updates = patrol_deltas[patrol_deltas['dml_verb'] == 'U'][sql_cols]

In [ ]:
patrol_updates.head()

In [ ]:
sql_update(patrol_updates, 'tbl_dpr_patrol', engine, ['encounter_timestamp', 'site_id'])